In [31]:
import pdfplumber
from pathlib import Path
import pandas as pd
import re

# Rutas
pdf_path = Path('./data/raw/')
output_path = Path('./data/processed/fragmentos_explorados.csv')

# Acumulador
fragmentos = []

# Clasificación y extracción de número de artículo
def analizar_fragmento(frag):
    texto = frag.strip()
    texto_l = texto.lower()

    tipo = "Otro"
    numero_articulo = None

    if texto_l.startswith("considerando") or "la asamblea legislativa" in texto_l:
        tipo = "Preámbulo"
    elif texto_l.startswith("capítulo") or texto_l.startswith("sección"):
        tipo = "Capítulo"
    elif texto_l.startswith("art.") or texto_l.startswith("artículo"):
        tipo = "Artículo"
        match = re.search(r'art(?:ículo)?\.?\s*(\d+)', texto_l)
        if match:
            numero_articulo = int(match.group(1))
        if "definiciones" in texto_l:
            tipo = "Definiciones"
    elif "firma electrónica" in texto_l or "mensaje de datos" in texto_l:
        tipo = "Disposición específica"

    return tipo, numero_articulo

In [32]:
# Iterar sobre todos los PDFs
pdf_files = list(pdf_path.glob('*.pdf'))
assert pdf_files, "No se encontraron PDFs en ./data/raw/"

for pdf_file in pdf_files:
    print(f"Procesando: {pdf_file.name}")
    with pdfplumber.open(pdf_file) as pdf:
        pages = [page.extract_text() for page in pdf.pages]
    
    raw_text = ' '.join(p for p in pages if p).replace('\r', '').replace('\n', ' ').strip()

    # Dividir por "Art. n" o "Artículo n"
    bloques = re.split(r'(?=Art\.?\s*\d+|Artículo\s+\d+)', raw_text)
    bloques = [b.strip() for b in bloques if len(b.strip()) > 50]

    for b in bloques:
        tipo, n_art = analizar_fragmento(b)
        fragmentos.append({
            "documento": pdf_file.name,
            "tipo_fragmento": tipo,
            "numero_articulo": n_art,
            "texto": b
        })

Procesando: BAA0E931-E117-487D-8900-A1D9876D5FBE.pdf
Procesando: DC5854_Reglamento aplicacion Codigo Tributario.pdf
Procesando: Catálogos- Sistema de Transmisión - 1.1.pdf
Procesando: 700-DGII-MN-2021-26031.pdf
Procesando: Reformas-DTE-D.O.-20-09-2022.pdf
Procesando: Manual de Acreditamiento y obtención de certificado de firma electrónica.pdf


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
Cannot set gray non-stroke color because /'Pattern4' is an invalid float value


Procesando: DC9226_Ley_del_Impuesto_a_la_Transferencia_de_Bienes_Muebles_y_a_la_Prestacion_de_Servicios.pdf
Procesando: Ley_de_Firma_Electrónica.pdf
Procesando: Normativa de Cumplimiento de los Documentos Tributarios Electrónicos_Versión 1.0_15022023.pdf
Procesando: Guía del Proceso de Incorporación para Ser Emisor De Documentos Tributarios Electrónicos.pdf


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value


Procesando: Manual de Usuario de la Consulta Pública de los DTE.pdf
Procesando: Guía rápida para emitir una Factura en la plataforma Sistema de Facturación.pdf
Procesando: Manual Técnico para la Integración Tecnológica del Sistema de Transmisión.pdf
Procesando: Aviso mandatario referente a ventas con factura.pdf
Procesando: Guia rápida emitir un CCF en la plataforma Sistema de Facturación.pdf
Procesando: Manual de solicitud en línea para ser autorizado como Emisor de DTE.pdf
Procesando: Normativa de Cumplimiento de los Documentos Tributarios Electrónicos_Versión 1.0_15022023 (1).pdf
Procesando: Manual funcional del Sistema de Transmisión.pdf


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value


Procesando: Manual del usuario para la Solicitud de Ingreso al ambiente para pruebas.pdf
Procesando: 297557B6-E5FD-4A41-B734-089751C41FDA.pdf
Procesando: Manual de Usuario del Sitio de Emisores DTE.pdf


Cannot set gray non-stroke color because /'Pattern1' is an invalid float value
Cannot set gray non-stroke color because /'Pattern2' is an invalid float value
Cannot set gray non-stroke color because /'Pattern3' is an invalid float value
Cannot set gray non-stroke color because /'Pattern4' is an invalid float value


Procesando: Manual de Usuario del Sistema de Facturación.pdf


In [33]:
# Guardar resultado
df = pd.DataFrame(fragmentos)
output_path.parent.mkdir(parents=True, exist_ok=True)
df.to_csv(output_path, index=False)
print(f"\n✅ Guardado en: {output_path}")


✅ Guardado en: data/processed/fragmentos_explorados.csv
